In [1]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Input,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import mlflow
from mlflow.models.signature import infer_signature
import joblib

2025-07-25 09:42:23.142924: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-25 09:42:23.143831: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-25 09:42:23.148160: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-25 09:42:23.159206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753432943.179793    5722 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753432943.18

In [2]:
sentence=['the glass of milk',
     'the glass of juice',
     'the cup of tea',
    'I am a good boy',
     'I am a good developer',
     'understand the meaning of words',
     'your videos are good',]

In [3]:
def transform_sentence_vector (sentence):
    return [one_hot(phrase,10000) for phrase in sentence]


In [4]:
sentence_embendding=transform_sentence_vector(sentence)
sentence_embendding

[[8737, 830, 299, 1557],
 [8737, 830, 299, 7119],
 [8737, 6419, 299, 1150],
 [1729, 930, 9010, 9840, 3002],
 [1729, 930, 9010, 9840, 3983],
 [4409, 8737, 8967, 299, 3845],
 [7337, 4729, 9120, 9840]]

In [5]:
def transform_vector_padding (sentence_embendding):
    return pad_sequences(sentence_embendding,padding="pre",maxlen=8)


In [6]:
sentence_embendding_padding=transform_vector_padding(sentence_embendding)
sentence_embendding_padding

array([[   0,    0,    0,    0, 8737,  830,  299, 1557],
       [   0,    0,    0,    0, 8737,  830,  299, 7119],
       [   0,    0,    0,    0, 8737, 6419,  299, 1150],
       [   0,    0,    0, 1729,  930, 9010, 9840, 3002],
       [   0,    0,    0, 1729,  930, 9010, 9840, 3983],
       [   0,    0,    0, 4409, 8737, 8967,  299, 3845],
       [   0,    0,    0,    0, 7337, 4729, 9120, 9840]], dtype=int32)

In [7]:
model=Sequential()
model.add(Embedding(input_dim=10000, output_dim=10)) 
model.compile(optimizer="adam", loss="mse") 

E0000 00:00:1753432947.183576    5722 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1753432947.187406    5722 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
def model_tracking():
    mlflow.set_tracking_uri("http://localhost:5000/")
    mlflow.set_experiment("word_embending")
    with mlflow.start_run():
        mlflow.log_param("input_dim",10000)
        mlflow.log_param("output_dim",10)
        mlflow.log_param("input_length",6)
        signature=infer_signature( model_input=sentence_embendding_padding,model_output=model.predict(sentence_embendding_padding))
        mlflow.keras.log_model(model,name="model",signature=signature)

In [10]:
model_tracking

<function __main__.model_tracking()>

In [11]:
sentence_embendding_padding[0]

array([   0,    0,    0,    0, 8737,  830,  299, 1557], dtype=int32)

In [12]:
model.predict(sentence_embendding_padding)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step


array([[ 0.04007879,  0.02619538, -0.03875845, -0.03721585, -0.03615486,
         0.00368946,  0.01702715, -0.01268416,  0.00941866,  0.02426061],
       [ 0.04007879,  0.02619538, -0.03875845, -0.03721585, -0.03615486,
         0.00368946,  0.01702715, -0.01268416,  0.00941866,  0.02426061],
       [ 0.04007879,  0.02619538, -0.03875845, -0.03721585, -0.03615486,
         0.00368946,  0.01702715, -0.01268416,  0.00941866,  0.02426061],
       [ 0.04007879,  0.02619538, -0.03875845, -0.03721585, -0.03615486,
         0.00368946,  0.01702715, -0.01268416,  0.00941866,  0.02426061],
       [-0.03278907, -0.04585463, -0.03295406,  0.04402131, -0.04117459,
        -0.04283339,  0.02117208,  0.02302691, -0.03546295, -0.03174757],
       [-0.02797508,  0.03726116, -0.02764305, -0.02085444, -0.02588179,
         0.01762413, -0.00251118, -0.00458878,  0.00500754, -0.04935205],
       [ 0.03710473, -0.00938637, -0.03644222,  0.0226096 , -0.01807066,
        -0.03387954, -0.01370735, -0.01116222

In [13]:
joblib.dump(model, 'model.pkl')

['model.pkl']